In [3]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
from scipy.interpolate import interp1d
from scipy.stats import entropy, iqr, kurtosis, mode, skew
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (Activation, BatchNormalization, Bidirectional,
                                     Conv1D, Conv2D, Dense, Dropout,
                                     Flatten, GlobalAveragePooling1D, LSTM,
                                     MaxPool1D, MaxPooling2D, Reshape,
                                     TimeDistributed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import pickle
from pandas import read_csv, unique
from scipy.signal import argrelextrema, butter, find_peaks, lfilter, lfilter_zi
from tensorflow import keras
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from kerastuner import RandomSearch, Hyperband
import kerastuner


/var/folders/ls/rcj43y9j0d38d39d28pbbfz40000gn/T/ipykernel_16358/4238391245.py:29: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch, Hyperband


In [4]:

X_respeck = pickle.load(open("X_respeck_all_phys", "rb"))
y_respeck = pickle.load(open("y_respeck_all_phys", "rb"))
print(X_respeck.shape)
print(y_respeck.shape)
print(y_respeck[0])
enc = OneHotEncoder(handle_unknown='ignore')
enc_fitted = enc.fit(y_respeck)
enc_fitted.categories_
y_respeck = enc.transform(y_respeck).toarray()
print(enc_fitted.categories_)# One Hot is inexed as this array prints!!!!!

FileNotFoundError: [Errno 2] No such file or directory: 'X_respeck_all_phys'

In [103]:
X_train, X_temp, y_train, y_temp = train_test_split(X_respeck, y_respeck, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# enc = enc.fit(np.array(y_train))
# y_train = enc.transform(y_train)
# y_val = enc.transform(y_val)
# y_test = enc.transform(y_test)
# print(X_train.shape)
# print(y_val.shape)
# enc.categories_
X_train.shape


<class 'numpy.float64'>
<class 'numpy.float64'>


In [64]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units1', min_value=16, max_value=256, step=16), return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
    model.add(LSTM(units=hp.Int('lstm_units2', min_value=16, max_value=256, step=16), return_sequences=True, activation='relu'))
    model.add(Conv1D(filters=hp.Int('conv1d_filters1', min_value=16, max_value=128, step=16), kernel_size=hp.Int('conv1d_kernel_size1', min_value=2, max_value=5), activation='relu', strides=2))
    model.add(MaxPool1D(pool_size=hp.Int('maxpool1d_pool_size', min_value=2, max_value=5), padding='same'))
    model.add(Conv1D(filters=hp.Int('conv1d_filters2', min_value=16, max_value=128, step=16), kernel_size=hp.Int('conv1d_kernel_size2', min_value=2, max_value=5), activation='relu', strides=1))
    model.add(GlobalAveragePooling1D())
    model.add(BatchNormalization(epsilon=1e-06))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile the model with tunable hyperparameters
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss=categorical_crossentropy,
        metrics=['categorical_accuracy']
    )
    
    return model


In [65]:

tuner = Hyperband(
    build_model,
    factor=3,  # Reduction factor for the number of models in each bracket
    max_epochs=20,  # Maximum number of training epochs for each model
    objective='categorical_accuracy',  # The metric to optimize
    directory='tuner_dir',  # Directory to store results
    project_name='Task_1_larger_lstm', # Name of the tuning project
)


In [16]:
# from kerastuner.tuners import BayesianOptimization
# tuner = RandomSearch(
#     build_model,
#     objective='categorical_accuracy',  # The metric to optimize
#     max_trials=10,  # Number of trials
#     # num_initial_points=5,  # Number of initial random trials
#     directory='tuner_dir',  # Directory to store results
#     project_name='cnn_lstm_random_no_max_pool'  # Name of the tuning project
# )


In [66]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), batch_size=64, shuffle=False)

Trial 30 Complete [00h 23m 09s]
categorical_accuracy: 0.9372983574867249

Best categorical_accuracy So Far: 0.9419420957565308
Total elapsed time: 04h 52m 36s
INFO:tensorflow:Oracle triggered exit


In [67]:
tuner.get_best_hyperparameters()[0].values
best = tuner.get_best_models()[0]
best.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 192)           155904    
                                                                 
 lstm_1 (LSTM)               (None, 50, 224)           373632    
                                                                 
 conv1d (Conv1D)             (None, 24, 96)            86112     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 8, 96)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 96)             27744     
                                                                 
 global_average_pooling1d (G  (None, 96)               0         
 lobalAveragePooling1D)                                 

In [68]:
with open('Task1.json', 'w') as f:
    f.write(best.to_json())
best.save('Trained_Task1_2.keras')

In [95]:
trained_t1 = keras.models.load_model('Trained_Task1_2.keras')

In [ ]:
# cnn_lstm = Sequential()
# cnn_lstm.add(LSTM(122, return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
# cnn_lstm.add(LSTM(128,return_sequences=True, activation='relu'))
# cnn_lstm.add(Conv1D(filters=64,kernel_size=3, activation='relu', strides=2))
# cnn_lstm.add(MaxPool1D(pool_size=5, padding='same'))
# cnn_lstm.add(Conv1D(filters=112, kernel_size=3, activation='relu', strides=1))
# cnn_lstm.add(GlobalAveragePooling1D())
# cnn_lstm.add(BatchNormalization(epsilon=1e-06))
# cnn_lstm.add(Dense(12))
# cnn_lstm.add(Activation('softmax'))
# cnn_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# cnn_lstm.fit(X_train, y_train, batch_size=64, verbose=1, validation_data=(X_val, y_val), epochs=50)

In [70]:
best.fit(X_train, y_train, batch_size=64, verbose=1, validation_data=(X_val, y_val), epochs=50)

Epoch 1/50
717/717 [==============================] - 131s 179ms/step - loss: 0.1610 - categorical_accuracy: 0.9294 - val_loss: 0.2115 - val_categorical_accuracy: 0.9196
Epoch 2/50
717/717 [==============================] - 128s 179ms/step - loss: 0.1420 - categorical_accuracy: 0.9365 - val_loss: 0.1686 - val_categorical_accuracy: 0.9262
Epoch 3/50
717/717 [==============================] - 128s 179ms/step - loss: 0.1385 - categorical_accuracy: 0.9397 - val_loss: 0.1653 - val_categorical_accuracy: 0.9287
Epoch 4/50
717/717 [==============================] - 129s 180ms/step - loss: 0.1368 - categorical_accuracy: 0.9393 - val_loss: 0.1626 - val_categorical_accuracy: 0.9302
Epoch 5/50
717/717 [==============================] - 137s 191ms/step - loss: 0.1285 - categorical_accuracy: 0.9427 - val_loss: 0.1580 - val_categorical_accuracy: 0.9283
Epoch 6/50
717/717 [==============================] - 127s 177ms/step - loss: 0.1254 - categorical_accuracy: 0.9440 - val_loss: 0.1550 - val_categoric

In [109]:
with open('Task1.json', 'w') as f:
    f.write(best.to_json())
best.save('Trained_Task1_2.keras')

In [110]:
trained_t1 = keras.models.load_model('Trained_Task1_2.keras')

In [73]:
best.evaluate(X_test, y_test)

180/180 [==============================] - 8s 43ms/step - loss: 0.0982 - categorical_accuracy: 0.9658


[0.09815269708633423, 0.9658119678497314]

In [107]:
converter = tf.lite.TFLiteConverter.from_keras_model(best)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# Save the model.
with open('APP_READY_cnn_lstm_physical.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpds1hy76v/assets


INFO:tensorflow:Assets written to: /tmp/tmpds1hy76v/assets
2023-11-03 18:23:00.381609: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-11-03 18:23:00.381652: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-11-03 18:23:00.382245: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpds1hy76v
2023-11-03 18:23:00.388949: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-11-03 18:23:00.388991: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpds1hy76v
2023-11-03 18:23:00.423583: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-11-03 18:23:00.546188: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpds1hy76v
2023-11-03 18:23:00.593967: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

In [1]:
tasks = ['ascending stairs', 'descending stairs', 'lying down back',
       'lying down on left', 'lying down on stomach', 'lying down right',
       'miscellaneous movements', 'normal walking', 'running',
       'shuffle walking', 'sitting', 'standing']
X_respeck_stationary = []
y_respeck_stationary = []
for i in range(len(X_respeck)):
    if not tasks[np.argmax(y_respeck[i])] in ["normal walking", "shuffle walking","running", "miscellaneous movements", "ascending stairs", "descending stairs"]:
        X_respeck_stationary.append(X_respeck[i])
        y_respeck_stationary.append(np.array([tasks[np.argmax(y_respeck[i])]]))
        
X_respeck_2 = np.array(X_respeck_stationary)
y_respeck_2 = np.array(y_respeck_stationary)


NameError: name 'X_respeck' is not defined

In [105]:
X_train, X_temp, y_train, y_temp = train_test_split(X_respeck_2, y_respeck_2, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print(np.unique(y_train))
y_train = enc.transform(y_train)
y_val = enc.transform(y_val)
y_test = enc.transform(y_test)
print(y_train.shape)

['lying down back' 'lying down on left' 'lying down on stomach'
 'lying down right' 'sitting' 'standing']
(39614, 12)


In [113]:
trained_t1.evaluate(X_test, y_test.toarray())

155/155 [==============================] - 8s 43ms/step - loss: 0.0417 - categorical_accuracy: 0.9838


[0.04170555993914604, 0.9838449358940125]